# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 8 2022 2:34PM,241443,22,620279,"NBTY Global, Inc.",Released
1,Jul 8 2022 2:33PM,241442,22,620274,"NBTY Global, Inc.",Released
2,Jul 8 2022 2:30PM,241441,22,620280,"NBTY Global, Inc.",Released
3,Jul 8 2022 2:23PM,241440,10,MSP211989,"Methapharm, Inc.",Released
4,Jul 8 2022 2:23PM,241440,10,MSP211991,"Methapharm, Inc.",Released
5,Jul 8 2022 2:23PM,241440,10,MSP211992,"Methapharm, Inc.",Released
6,Jul 8 2022 2:05PM,241439,16,8248034,Sartorius Bioprocess Solutions,Released
7,Jul 8 2022 2:05PM,241439,16,8248079,Sartorius Bioprocess Solutions,Released
8,Jul 8 2022 2:05PM,241439,16,8248617,Sartorius Bioprocess Solutions,Released
9,Jul 8 2022 2:05PM,241439,16,8248619,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
60,241439,Released,6
61,241440,Released,3
62,241441,Released,1
63,241442,Released,1
64,241443,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241439,NaN,NaN,6.0
241440,NaN,NaN,3.0
241441,NaN,NaN,1.0
241442,NaN,NaN,1.0
241443,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241348,0.0,0.0,1.0
241351,0.0,1.0,0.0
241360,0.0,0.0,2.0
241361,0.0,0.0,1.0
241362,0.0,0.0,17.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241348,0,0,1
241351,0,1,0
241360,0,0,2
241361,0,0,1
241362,0,0,17


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241348,0,0,1
1,241351,0,1,0
2,241360,0,0,2
3,241361,0,0,1
4,241362,0,0,17


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241348,,,1
1,241351,,1,
2,241360,,,2
3,241361,,,1
4,241362,,,17


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc."
1,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc."
2,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc."
3,Jul 8 2022 2:23PM,241440,10,"Methapharm, Inc."
6,Jul 8 2022 2:05PM,241439,16,Sartorius Bioprocess Solutions
10,Jul 8 2022 2:05PM,241439,16,Sartorius Lab Products and Service
12,Jul 8 2022 1:38PM,241438,19,"GCH Granules USA, Inc."
13,Jul 8 2022 1:14PM,241434,10,Emerginnova
14,Jul 8 2022 1:09PM,241432,10,"Senseonics, Incorporated"
15,Jul 8 2022 1:06PM,241431,16,Sartorius Bioprocess Solutions


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc.",,,1
1,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc.",,,1
2,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc.",,,1
3,Jul 8 2022 2:23PM,241440,10,"Methapharm, Inc.",,,3
4,Jul 8 2022 2:05PM,241439,16,Sartorius Bioprocess Solutions,,,6
5,Jul 8 2022 2:05PM,241439,16,Sartorius Lab Products and Service,,,6
6,Jul 8 2022 1:38PM,241438,19,"GCH Granules USA, Inc.",,,1
7,Jul 8 2022 1:14PM,241434,10,Emerginnova,,,1
8,Jul 8 2022 1:09PM,241432,10,"Senseonics, Incorporated",,,1
9,Jul 8 2022 1:06PM,241431,16,Sartorius Bioprocess Solutions,,1,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc.",1,,
1,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc.",1,,
2,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc.",1,,
3,Jul 8 2022 2:23PM,241440,10,"Methapharm, Inc.",3,,
4,Jul 8 2022 2:05PM,241439,16,Sartorius Bioprocess Solutions,6,,
5,Jul 8 2022 2:05PM,241439,16,Sartorius Lab Products and Service,6,,
6,Jul 8 2022 1:38PM,241438,19,"GCH Granules USA, Inc.",1,,
7,Jul 8 2022 1:14PM,241434,10,Emerginnova,1,,
8,Jul 8 2022 1:09PM,241432,10,"Senseonics, Incorporated",1,,
9,Jul 8 2022 1:06PM,241431,16,Sartorius Bioprocess Solutions,5,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc.",1,,
1,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc.",1,,
2,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc.",1,,
3,Jul 8 2022 2:23PM,241440,10,"Methapharm, Inc.",3,,
4,Jul 8 2022 2:05PM,241439,16,Sartorius Bioprocess Solutions,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc.",1.0,NaN,NaN
2,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jul 8 2022 2:23PM,241440,10,"Methapharm, Inc.",3.0,NaN,NaN
4,Jul 8 2022 2:05PM,241439,16,Sartorius Bioprocess Solutions,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 8 2022 2:34PM,241443,22,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jul 8 2022 2:33PM,241442,22,"NBTY Global, Inc.",1.0,0.0,0.0
2,Jul 8 2022 2:30PM,241441,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jul 8 2022 2:23PM,241440,10,"Methapharm, Inc.",3.0,0.0,0.0
4,Jul 8 2022 2:05PM,241439,16,Sartorius Bioprocess Solutions,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7724740,189.0,7.0,3.0
102,482848,2.0,0.0,0.0
15,241378,124.0,8.0,36.0
16,965682,17.0,2.0,0.0
18,482773,1.0,1.0,0.0
19,965636,2.0,8.0,29.0
20,724163,20.0,6.0,3.0
21,241348,1.0,0.0,0.0
22,724326,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7724740,189.0,7.0,3.0
1,102,482848,2.0,0.0,0.0
2,15,241378,124.0,8.0,36.0
3,16,965682,17.0,2.0,0.0
4,18,482773,1.0,1.0,0.0
5,19,965636,2.0,8.0,29.0
6,20,724163,20.0,6.0,3.0
7,21,241348,1.0,0.0,0.0
8,22,724326,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,189.0,7.0,3.0
1,102,2.0,0.0,0.0
2,15,124.0,8.0,36.0
3,16,17.0,2.0,0.0
4,18,1.0,1.0,0.0
5,19,2.0,8.0,29.0
6,20,20.0,6.0,3.0
7,21,1.0,0.0,0.0
8,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,189.0
1,102,Released,2.0
2,15,Released,124.0
3,16,Released,17.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,18,19,20,21,22
Status,,,,,,,,,
Completed,3.0,0.0,36.0,0.0,0.0,29.0,3.0,0.0,0.0
Executing,7.0,0.0,8.0,2.0,1.0,8.0,6.0,0.0,0.0
Released,189.0,2.0,124.0,17.0,1.0,2.0,20.0,1.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,18,19,20,21,22
0,Completed,3.0,0.0,36.0,0.0,0.0,29.0,3.0,0.0,0.0
1,Executing,7.0,0.0,8.0,2.0,1.0,8.0,6.0,0.0,0.0
2,Released,189.0,2.0,124.0,17.0,1.0,2.0,20.0,1.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,18,19,20,21,22
0,Completed,3.0,0.0,36.0,0.0,0.0,29.0,3.0,0.0,0.0
1,Executing,7.0,0.0,8.0,2.0,1.0,8.0,6.0,0.0,0.0
2,Released,189.0,2.0,124.0,17.0,1.0,2.0,20.0,1.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()